In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Periodic-table-final.csv')
data.head()

,Symbol,Atomic Number,Electronegativity,Atomic Radii (pm),Thermal Conductivity,Density,Crystal System
0,H,1,2.20,53.0,0.1805,0.000090,Simple Hexagonal
1,He,2,NaN,31.0,0.1513,0.000179,Face Centered Cubic
2,Li,3,0.98,167.0,85.0000,0.534000,Body Centered Cubic
3,Be,4,1.57,112.0,190.0000,1.850000,Simple Hexagonal
4,B,5,2.04,87.0,27.0000,2.340000,Simple Trigonal


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symbol                118 non-null    object 
 1   Atomic Number         118 non-null    int64  
 2   Electronegativity     96 non-null     float64
 3   Atomic Radii (pm)     86 non-null     float64
 4   Thermal Conductivity  94 non-null     float64
 5   Density               105 non-null    float64
 6   Crystal System        113 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 6.6+ KB
